In [1]:
import spacy
import re
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import numpy as np
import os
import json
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [2]:
#with open('data/amitheasshole_corpus.json', 'r', encoding='UTF-8') as f:
#    text = f.read()
    
#print(text)

In [3]:
with open('data/amitheasshole_corpus.json') as json_file:
    data = json.load(json_file)
    
print("Type:", type(data))
for values, i in enumerate(data):
    print(i)
    if values>500:
        break

Type: <class 'list'>
{'author': 'BouyHowdy', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_3dskjw4m', 'author_patreon_flair': False, 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1554076844, 'domain': 'self.AmItheAsshole', 'full_link': 'https://www.reddit.com/r/AmItheAsshole/comments/b7umhd/aita_for_not_wanting_to_be_involved_with_my_milfil/', 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0}, 'id': 'b7umhd', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': True, 'is_self': True, 'is_video': False, 'link_flair_background_color': '', 'link_flair_richtext': [], 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follow': True, 'num_comments': 16, 'num_crossposts': 0, 'over_18': False, 'parent_whitelist_status': 'all_ads', 'permalink': '/r/AmItheAsshole/c

In [29]:
print(type(data[0]))

<class 'dict'>


In [30]:
print("\nAuthor:", data[0]['author'])


Author: BouyHowdy


In [31]:
print("\nContent:", data[0]['selftext'])
print("\nID:", data[0]['id'])


Content: Alright, so my fiance and I had a baby together a couple moths ago.

Before then, his parents didn't necessarily mind if they saw us or not. In an entire year, they invited us to 2 events total, not even necessarily holidays. Not only that, but they always treat my fiance as the black sheep of the family. When he was younger, his mom gave his dad an ultimatum and they ended up sending him to a private school.

Since our baby's birth, his parents have been wanting to spend time with her. I don't mind. I let them have their time with her and she even gets to spend the night with her grandparents if they so choose.

However, they don't necessarily attempt to make my fiance and I feel welcome. They will make small talk, but other than that, it feels as though they want us to leave so they can have their time with the baby.

On top of that, whenever I suggest something for them to do with her, they always act like it's wrong. Our baby has to go to physical therapy for a common tig

In [4]:
dicts = {}
for values, i in enumerate(data):
    dicts[data[values]['id']] = data[values]['selftext']
    #print(values)
    if values>500:
        break
        
print(dicts)

{'b7umhd': "Alright, so my fiance and I had a baby together a couple moths ago.\n\nBefore then, his parents didn't necessarily mind if they saw us or not. In an entire year, they invited us to 2 events total, not even necessarily holidays. Not only that, but they always treat my fiance as the black sheep of the family. When he was younger, his mom gave his dad an ultimatum and they ended up sending him to a private school.\n\nSince our baby's birth, his parents have been wanting to spend time with her. I don't mind. I let them have their time with her and she even gets to spend the night with her grandparents if they so choose.\n\nHowever, they don't necessarily attempt to make my fiance and I feel welcome. They will make small talk, but other than that, it feels as though they want us to leave so they can have their time with the baby.\n\nOn top of that, whenever I suggest something for them to do with her, they always act like it's wrong. Our baby has to go to physical therapy for a 

In [55]:
entity_text_dict = {}
for i in dicts: 
    doc = nlp(dicts[i].replace('\n',''))
    entitylist = []
    for token in doc.ents:
        entitylist.append(token.text)
        
    entity_text_dict[i] = entitylist
    
print(entity_text_dict)

{'b7umhd': ['a couple moths ago', 'an entire year', '2', 'WIBTA'], 'b7uqbc': ['this weekend', 'asian', 'asian', 'asian', 'two', 'asian', 'Texas', 'texas', 'asian', 'english', 'asian', 'AITA'], 'b7uql4': ['Saturday', 'last minute', 'Mcdonald', '11h30', '18 months old', 'Some minutes later', '3', 'three', 'two weeks', 'Asshole'], 'b7ur4w': [], 'b7urfq': ['over 600 miles', 'Uncle', 'the week', 'Grandma', 'Grandpa', 'Monday', 'wear &amp', 'this week', 'next week', 'a full week', 'Grandpa', 'the week', 'the weekend'], 'b7usq5': ['one', 'today'], 'b7usr7': ['WIBTA', '2 days ago'], 'b7ustg': ['11 years', '13', '11', 'first', 'the years', 'two', 'two weeks ago', 'one'], 'b7uugl': ['the last few years', 'Today', 'this day'], 'b7uuqj': ['4 years', 'the last 4 years', '2 months', '100%', 'AITA', 'Ex', 'the last 3ish months', 'AF', '1', 'Ex', 'W.'], 'b7uux6': [], 'b7uwjg': [], 'b7uxny': ['Bible'], 'b7uyy5': ['First', 'today', 'two', 'today', 'first'], 'b7uz0r': [], 'b7uz2k': ['20', '4 and a half y

In [ ]:
for span in doc.sents:
    blob = TextBlob(span.text, analyzer=NaiveBayesAnalyzer())
    print(span, blob.sentiment)
    print('')

In [70]:
sentiment_text_dict = {}
for values, i in enumerate(dicts): 
    doc = nlp(dicts[i].replace('\n',''))
    #print(values)
    for span in doc.sents:
        blob = TextBlob(span.text, analyzer=NaiveBayesAnalyzer())
        sentiment_text_dict[i] = blob.sentiment
    if values > 5:
        break
    
print(sentiment_text_dict)

0
Alright, so my fiance and I had a baby together a couple moths ago.

Before then, his parents didn't necessarily mind if they saw us or not. In an entire year, they invited us to 2 events total, not even necessarily holidays. Not only that, but they always treat my fiance as the black sheep of the family. When he was younger, his mom gave his dad an ultimatum and they ended up sending him to a private school.

Since our baby's birth, his parents have been wanting to spend time with her. I don't mind. I let them have their time with her and she even gets to spend the night with her grandparents if they so choose.

However, they don't necessarily attempt to make my fiance and I feel welcome. They will make small talk, but other than that, it feels as though they want us to leave so they can have their time with the baby.

On top of that, whenever I suggest something for them to do with her, they always act like it's wrong. Our baby has to go to physical therapy for a common tight muscl

5
So me and my brother went out for a smoke session before going out to eat. What we usually do is that while someone is driving the other one smokes but today i got too high therefore i was not able to drive, so i told him and he was telling that i was being selfish for not letting him smoke, but then i convinced him of driving me to a place where i would feel safe and familiar so he did. When we got there i started driving to our destination(which was not far, it was less than 2-3 miles) and he started smoking. Halfway to where we were going i told him that we should stop and rest as i saw that he was really high and i was really fucking high. He then started saying that he was not high and that if i would not drive he would drive but we were both stoned. I then call my other brother and he came and got us out of there. Was i the asshole for getting really high, not wanting to drive and believing that my brother was high so i did not let him drive? Or is he the asshole for pushing me

In [11]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

[W 2023-01-27 16:19:07.502 LabApp] 'iopub_data_rate_limit' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2023-01-27 16:19:07.502 LabApp] 'iopub_data_rate_limit' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[I 2023-01-27 16:19:07.506 LabApp] JupyterLab extension loaded from /home/somber/anaconda3/lib/python3.9/site-packages/jupyterlab
[I 2023-01-27 16:19:07.507 LabApp] JupyterLab application directory is /home/somber/anaconda3/share/jupyter/lab
[I 16:19:07.509 NotebookApp] The port 8888 is already in use, trying another port.
[I 16:19:07.510 NotebookApp] Serving notebooks from local directory: /home/somber/repos/DMGK-NLP
[I 16:19:07.510 NotebookApp] Jupyter Notebook 6.4.12 is running at:
[I 16:19:07.510 NotebookApp] http://localhost:8889/?token=a226cf1bb41246699b4cb03915aeb03f2f8ae542560c7968
[I 16:19:07

In [16]:
tokenized_corpus = []
extra_stop = ['mr', 'ms', 'mrs', 'hon']
text_proc = []
for values, i in enumerate(dicts): 
    doc = nlp(dicts[i].replace('\n',''))
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num and token.lemma_.lower() not in extra_stop:
            text_proc.append(token.lemma_.lower())
    tokenized_corpus.append(text_proc)
    #if values>50:
     #   break
#print(tokenized_corpus)

In [18]:
words_id = corpora.Dictionary(tokenized_corpus)
corpus = [words_id.doc2bow(txt) for txt in tokenized_corpus]
k_init = 5
k_final = 15
for k in range(k_init,k_final+1):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=words_id, num_topics = k, random_state=50, passes=20, per_word_topics=True)

    per_lda = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=words_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(k,per_lda,coherence_lda)

5 -7.122510946390525 0.23315964708127965
6 -7.123666996124174 0.2354447076312229


KeyboardInterrupt: 